# テキストデータの可視化

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_5/intro-to-visualizing-text-data.ipynb)

今回は、[青空文庫](https://www.aozora.gr.jp/)に収録されている、夏目漱石の[『三四郎』](https://www.aozora.gr.jp/cards/000148/card794.html)という作品を使って、テキストデータの可視化を練習します。

## 基礎的な解析

In [ ]:
# ライブラリのインストール

!pip install ginza==4.0.6 ja-ginza==4.0.0
!pip install sklearn pandas

### 形態素解析

### 係り受け解析

### 分類

### クラスタリング

## 可視化